<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Papers/blob/main/P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import ternsorflow as tf